# Spectroscopy

## Infrared
We will compute spectra for Alanine in this notebook. If your computer is slow, just change the SMILES code to something small, e.g. formaldehyde 'C=O'.

In [ ]:
import veloxchem as vlx
import time
start_time = time.time()

In [ ]:
molecule = vlx.Molecule.read_smiles('C[C@@H](C(=O)O)N')
molecule.show(atom_indices=True)

We start by optimizing the molecule.

In [ ]:
basis = vlx.MolecularBasis.read(molecule, 'def2-sv(p)')
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = 'blyp'
scf_drv.ri_coulomb = True
scf_drv.dispersion = True
scf_drv.grid_level = 2
scf_drv.conv_thresh = 1e-03
results = scf_drv.compute(molecule, basis)
resp_drv = vlx.RespChargesDriver()
charges = resp_drv.compute(molecule, basis, results, 'resp')
conf_generator = vlx.ConformerGenerator()
conf_generator.resp_charges = False
conf_generator.partial_charges = charges
conformers = conf_generator.generate(molecule)
molecule = conformers['molecules'][0]
basis = vlx.MolecularBasis.read(molecule, 'def2-SVP')
opt_drv = vlx.OptimizationDriver(scf_drv)
opt_drv.conv_energy = 1e-04
opt_drv.conv_drms = 1e-02
opt_drv.conv_dmax = 2e-02
opt_drv.conv_grms = 4e-03
opt_drv.conv_gmax = 8e-03
opt_results = opt_drv.compute(molecule, basis, results)

We have a function to display the convergence and the corresponding geometries.

In [ ]:
opt_drv.show_convergence(opt_results)

Now we read in the final geometry from the optimization and compute the vibrational analysis using the ```.VibrationalAnalysis()```class. 

In [ ]:
molecule2 = vlx.Molecule.read_xyz_string(opt_results['final_geometry'])
basis2 = vlx.MolecularBasis.read(molecule2, 'def2-sv(p)')
scf_drv2 = vlx.ScfRestrictedDriver()
scf_drv2.xcfun = 'b3lyp'
results2 = scf_drv2.compute(molecule2, basis2)
vibanalysis_drv = vlx.VibrationalAnalysis(scf_drv2)
vib_results = vibanalysis_drv.compute(molecule2, basis2)

The results could either be accessed as arrays...

In [ ]:
print(vibanalysis_drv.vib_frequencies)

In [ ]:
print(vibanalysis_drv.ir_intensities)

... or we can use the build in plot function to show the spectra and the animate function to visualize a specific mode. (If you have another molecule than alanine just exchange the mode=26 for another mode)

In [ ]:
vibanalysis_drv.plot_ir(vib_results, invert_axes=True)

In [ ]:
vibanalysis_drv.animate(vib_results, mode=31)

In [ ]:
vibanalysis_drv.gibbs_free_energy

## UV/vis

To compute UV/Vis spectra you call the linear response solver. We use the molecule, basis set and scf results object from the IR computation.

In [ ]:
rpa_solver = vlx.lreigensolver.LinearResponseEigenSolver()
rpa_solver.update_settings({"nstates": 4})
rpa_results = rpa_solver.compute(molecule2, basis2, results2)
print("Time: " + str(time.time() - start_time) + " seconds")

The results can be visualized as plots. The response solver also computes the rotatory strengths and we can plot the ECD spectrum with the plot_ecd function.

In [ ]:
rpa_solver.plot_uv_vis(rpa_results)

In [ ]:
rpa_solver.plot_ecd(rpa_results)